In [2]:
!pip install -q streamlit

In [3]:
!pip install -q streamlit_jupyter

In [6]:
!pip install yahoo_fin

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for yahoo_fin from https://files.pythonhosted.org/packages/c1/73/b8d385bbc01bacb66f589a36f6c108e0e847d1f982755bfcc7391735d560/yahoo_fin-0.8.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for requests-html from https://files.pythonhosted.org/packages/24/bc/a4380f09bab3a776182578ce6b2771e57259d0d4dbce178205779abdc347/requests_html-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for feedparser from https://files.pythonhosted.org/packages/7c/d4/8c31aad9cc18f451c49f7f9cfb5799dadffc88177f7917bc90a66459b1d7/feedparser-6.0.11-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for pyquery from https://files.pythonhosted.org/packages/36/b7/f7ccf9e52e2817e1265d3719c600fa4ef33c07de4d5ef0ced3f43ab1cef2/pyquery-2.0.0-py3-none-any.whl.metadat

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
%run findash_app.py

2024-05-10 10:06:10.594 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-05-10 10:06:11.699 
  command:

    streamlit run findash_app.py [ARGUMENTS]


In [14]:
!python -m streamlit run findash_app.py

^C


In [18]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm
StreamlitPatcher().jupyter()

In [19]:
import yahoo_fin.stock_info as si
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#import pyfolio as pf

In [32]:
def tab1(ticker):
    
    st.title("Summary")
    st.write("Select ticker on the left to begin")
    st.write(ticker)
    
    #The code below gets the quota table from Yahoo Finance. The streamlit page
    #is divided into 2 columns and selected columns are displayed on each side of the page.

    def getsummary(ticker):
            table = si.get_quote_table(ticker, dict_result = False)
            return table 
        
    c1, c2 = st.columns((1,1))
    with c1:        
        if ticker != '-':
            summary = getsummary(ticker)
            summary['value'] = summary['value'].astype(str)
            showsummary = summary.iloc[[14, 12, 5, 2, 6, 1, 16, 3],]
            showsummary.set_index('attribute', inplace=True)
            st.dataframe(showsummary)
            
            
    with c2:        
        if ticker != '-':
            summary = getsummary(ticker)
            summary['value'] = summary['value'].astype(str)
            showsummary = summary.iloc[[11, 4, 13, 7, 8, 10, 9, 0],]
            showsummary.set_index('attribute', inplace=True)
            st.dataframe(showsummary)
            
                        
             
    #The code below uses the yahoofinance package to get all the available stock
    #price data. Plotly is then used to visualize the data.  An interesting feature
    #from plotly called range selector is also used. A list of dictionaries
    #is added to range selector to make buttons and identify the periods.
    #References:
    #https://plotly.com/python/range-slider/
    
        
    @st.cache 
    def getstockdata(ticker):
        stockdata = yf.download(ticker, period = 'MAX')
        return stockdata
        
    if ticker != '-':
            chartdata = getstockdata(ticker) 
                       
            fig = px.area(chartdata, chartdata.index, chartdata['Close'])
            
                     

            fig.update_xaxes(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1M", step="month", stepmode="backward"),
                        dict(count=3, label="3M", step="month", stepmode="backward"),
                        dict(count=6, label="6M", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1Y", step="year", stepmode="backward"),
                        dict(count=3, label="3Y", step="year", stepmode="backward"),
                        dict(count=5, label="5Y", step="year", stepmode="backward"),
                        dict(label = "MAX", step="all")
                    ])
                )
            )
            st.plotly_chart(fig)

In [33]:
ticker_list = ['-'] + si.tickers_sp500()

In [34]:
from yahoo_fin.stock_info import get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table

In [35]:
""" get list of all stocks currently in the S&P 500 """
sp500_ticker_list = ['AAPL']

In [36]:
global ticker
ticker = st.sidebar.selectbox("Select a ticker", ticker_list)

In [37]:
# Add a radio box
select_tab = st.sidebar.radio("Select tab", ['Summary', 'Chart', 'Statistics', 'Financials', 'Analysis', 'Monte Carlo Simulation', "Your Portfolio's Trend"])

In [38]:
tab1(ticker)

# Summary

Select ticker on the left to begin

-